In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
hyperparameters = {
    'num_classes': 10,
    'hidden_dim': 1024,
    'cnn_channels': [32, 64, 128, 256],
    'lr': 1e-3,
    'batch_size': 64,
    'val_split': 0.2,
    'test_split': 0.2,
    'num_workers': 4,
    'max_epochs': 10,
}

In [3]:
import pandas as pd
from dataset import SudokuDataModule

data = pd.read_csv('data/sudoku_small.csv')
dataset = SudokuDataModule(data, hyperparameters)

In [4]:
from model import SudokuSolverCNN
from lightning.pytorch.utilities.model_summary import ModelSummary

model = SudokuSolverCNN(hyperparameters)
ModelSummary(model, max_depth=2)

   | Name              | Type       | Params | Mode  | In sizes       | Out sizes     
--------------------------------------------------------------------------------------------
0  | cnn_layers        | Sequential | 387 K  | train | [1, 1, 9, 9]   | [1, 256, 9, 9]
1  | cnn_layers.conv_0 | Conv2d     | 320    | train | [1, 1, 9, 9]   | [1, 32, 9, 9] 
2  | cnn_layers.relu_0 | ReLU       | 0      | train | [1, 32, 9, 9]  | [1, 32, 9, 9] 
3  | cnn_layers.conv_1 | Conv2d     | 18.5 K | train | [1, 32, 9, 9]  | [1, 64, 9, 9] 
4  | cnn_layers.relu_1 | ReLU       | 0      | train | [1, 64, 9, 9]  | [1, 64, 9, 9] 
5  | cnn_layers.conv_2 | Conv2d     | 73.9 K | train | [1, 64, 9, 9]  | [1, 128, 9, 9]
6  | cnn_layers.relu_2 | ReLU       | 0      | train | [1, 128, 9, 9] | [1, 128, 9, 9]
7  | cnn_layers.conv_3 | Conv2d     | 295 K  | train | [1, 128, 9, 9] | [1, 256, 9, 9]
8  | cnn_layers.relu_3 | ReLU       | 0      | train | [1, 256, 9, 9] | [1, 256, 9, 9]
9  | fc_hidden         | Linear     |

In [5]:
from pytorch_lightning import Trainer
from pytorch_lightning.tuner.tuning import Tuner
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor

trainer = Trainer(
    max_epochs=10,
    logger=TensorBoardLogger('logs', name='sudoku-solver', log_graph=True),
    callbacks=[
        ModelCheckpoint(monitor='val_loss', save_top_k=1, mode='min'),
        EarlyStopping(monitor='val_loss', patience=5, mode='min'),
        LearningRateMonitor(logging_interval='step')
    ],
    enable_checkpointing=True,
    enable_progress_bar=True,
)
tuner = Tuner(trainer)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
tuner.lr_find(model, datamodule=dataset, max_lr=10, min_lr=1e-10, num_training=9375)

Train size: 600000
Val size: 200000
Test size: 200000


Finding best initial lr:   0%|          | 0/9375 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=9375` reached.
Learning rate set to 0.0045781849496414105
Restoring states from the checkpoint path at /home/atharva/Documents/SudokuSolver/.lr_find_6499c3c8-ae99-4c3c-9fcd-5ca1b2e77e24.ckpt
Restored all states from the checkpoint at /home/atharva/Documents/SudokuSolver/.lr_find_6499c3c8-ae99-4c3c-9fcd-5ca1b2e77e24.ckpt


In [7]:
trainer.fit(model, datamodule=dataset)


  | Name       | Type       | Params | Mode  | In sizes     | Out sizes     
----------------------------------------------------------------------------------
0 | cnn_layers | Sequential | 387 K  | train | [1, 1, 9, 9] | [1, 256, 9, 9]
1 | fc_hidden  | Linear     | 21.2 M | train | [1, 20736]   | [1, 1024]     
2 | fc_output  | Linear     | 830 K  | train | [1, 1024]    | [1, 810]      
----------------------------------------------------------------------------------
22.5 M    Trainable params
0         Non-trainable params
22.5 M    Total params
89.811    Total estimated model params size (MB)


Train size: 600000
Val size: 200000
Test size: 200000


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

/home/atharva/virtual_envs/ml_env/lib64/python3.12/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
trainer.test(model, datamodule=dataset)